# Booster Binary Classification Example

## Imports

In [1]:
import wallaroo
from wallaroo.deployment_config import DeploymentConfigBuilder
import pyarrow as pa
from wallaroo.framework import Framework

import pickle
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from xgboost import train, DMatrix

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://doc-test.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=MZEI-OLQU

Login successful!


### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [9]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

workspace_name = f'xgboost-booster-binary-classification'
pipeline_name = f'xgboost-booster-binary-classification'

model_name = 'xgboost-booster-binary-classification'
model_file_name = './models/xgb_booster_binary_classification.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = get_workspace(workspace_name, wl)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

## Configure & Upload Model

### Configure PyArrow Schema

In [7]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=30))
])

output_schema = pa.schema([
    pa.field('probabilities', pa.float32()),
])

### Upload model

You can find the model uploaded in [model zoo](https://storage.cloud.google.com/wallaroo-model-zoo/model-auto-conversion/convert-non-native-frameworks/xgboost/xgb_booster_binary_classification.pkl?authuser=0).

In [10]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime..
Ready


Name,xgboost-booster-binary-classification
Version,abf3c282-ad46-4a62-91e9-3eab998b2e21
File Name,xgb_booster_binary_classification.pkl
SHA,be6d8c51395196da4720951b45660840fecdd680e7d99003a966023873322d8b
Status,ready
Image Path,None
Architecture,None
Acceleration,None
Updated At,2024-20-Mar 21:52:58


In [11]:
model.config().runtime()

'onnx'

## Deploy Pipeline

In [12]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [13]:
pipeline_name = f"booster-binary-classification-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

Waiting for deployment - this will take up to 45s ............ ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.3.89',
   'name': 'engine-5d57dfb5d7-bvknx',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'booster-binary-classification-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'config': {'batch_config': None,
       'filter_threshold': None,
       'id': 12,
       'input_schema': '/////7AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAEAAAAyP///wAAARAUAAAAJAAAAAQAAAABAAAAMAAAAAYAAABpbnB1dHMAAAAABgAIAAQABgAAAB4AAAAQABQACAAGAAcADAAAABAAEAAAAAAAAQMQAAAAHAAAAAQAAAAAAAAABAAAAGl0ZW0AAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'model_version_id': 10,
       'output_schema': '/////4AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAUAAAAEAAUAAgABgAHAAwAAAAQABAAAAAAAAEDEAAAACQAAAAEAAAAAAAAAA0AAABwcm9iYWJpbGl0aWVzAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'runtime': 'onnx',
       'sidekick_uri': None,
       'tensor_fields

In [19]:
pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

Waiting for deployment - this will take up to 45s ............. ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.3.91',
   'name': 'engine-cff58d6f-wpz5z',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'booster-binary-classification-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'config': {'batch_config': None,
       'filter_threshold': None,
       'id': 12,
       'input_schema': '/////7AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAEAAAAyP///wAAARAUAAAAJAAAAAQAAAABAAAAMAAAAAYAAABpbnB1dHMAAAAABgAIAAQABgAAAB4AAAAQABQACAAGAAcADAAAABAAEAAAAAAAAQMQAAAAHAAAAAQAAAAAAAAABAAAAGl0ZW0AAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'model_version_id': 10,
       'output_schema': '/////4AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAUAAAAEAAUAAgABgAHAAwAAAAQABAAAAAAAAEDEAAAACQAAAAEAAAAAAAAAA0AAABwcm9iYWJpbGl0aWVzAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'runtime': 'onnx',
       'sidekick_uri': None,
       'tensor_fields':

## Run Inference

## Data

The dataset is from the `sklearn.datasets.load_breast_cancer` examples.  These are converted to a pandas DataFrame, that is submitted to the deployed model in Wallaroo for an inference request.

In [5]:
dataset = load_breast_cancer()
X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
dtrain = DMatrix(X_train, label=y_train)
dtest = DMatrix(X_test, label=y_test)

In [14]:
data = dtest.get_data().todense()[:100]

In [15]:
import pandas as pd

dataframe = pd.DataFrame({"inputs": data.tolist()})
dataframe

,inputs
0,"[12.470000267028809, 18.600000381469727, 81.08..."
1,"[18.940000534057617, 21.309999465942383, 123.5..."
2,"[15.460000038146973, 19.479999542236328, 101.6..."
3,"[12.399999618530273, 17.68000030517578, 81.470..."
4,"[11.539999961853027, 14.4399995803833, 74.6500..."
...,...
95,"[9.777000427246094, 16.989999771118164, 62.5, ..."
96,"[20.1299991607666, 28.25, 131.1999969482422, 1..."
97,"[13.8100004196167, 23.75, 91.55999755859375, 5..."
98,"[11.149999618530273, 13.079999923706055, 70.87..."


Getting results from the pipeline:

In [23]:
pipeline.infer(dataframe)

,time,in.inputs,out.probabilities,anomaly.count
0,2024-03-20 22:01:57.174,"[12.470000267, 18.6000003815, 81.0899963379, 4...",0.894763,0
1,2024-03-20 22:01:57.174,"[18.9400005341, 21.3099994659, 123.5999984741,...",0.107060,0
2,2024-03-20 22:01:57.174,"[15.4600000381, 19.4799995422, 101.6999969482,...",0.107060,0
3,2024-03-20 22:01:57.174,"[12.3999996185, 17.6800003052, 81.4700012207, ...",0.894763,0
4,2024-03-20 22:01:57.174,"[11.5399999619, 14.4399995804, 74.6500015259, ...",0.894763,0
...,...,...,...,...
95,2024-03-20 22:01:57.174,"[9.7770004272, 16.9899997711, 62.5, 290.200012...",0.894763,0
96,2024-03-20 22:01:57.174,"[20.1299991608, 28.25, 131.1999969482, 1261.0,...",0.107060,0
97,2024-03-20 22:01:57.174,"[13.8100004196, 23.75, 91.5599975586, 597.7999...",0.107060,0
98,2024-03-20 22:01:57.174,"[11.1499996185, 13.0799999237, 70.8700027466, ...",0.894763,0


## Undeploy Pipelines

In [24]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,booster-binary-classification-pipeline
created,2024-03-20 21:52:58.608666+00:00
last_updated,2024-03-20 21:54:02.874295+00:00
deployed,False
arch,None
accel,None
tags,
versions,"b5315104-0d42-48ad-903a-53dca70b64de, 9d7520b7-1504-4609-a2ff-9558bcd43ef4, 6f6cd619-8f31-40c7-b70a-d8e1c894b58d"
steps,xgboost-booster-binary-classification
published,False
